In [2]:
import requests
from datetime import datetime

In [3]:
def fetch_available_venue_timeslot(district_codes, facility_code, play_date):
    
    base_url = "https://www.smartplay.lcsd.gov.hk/rest/facility-catalog/api/v1/publ/facilities"

    params = {
        "distCode": ",".join(district_codes),
        "faCode": facility_code,
        "playDate": play_date
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0',
        'Referer': 'https://www.smartplay.lcsd.gov.hk/facilities/search-result',
        'channel': 'INTERNET'
    }

    response = requests.get(base_url, params=params, headers=headers)
    return response.json()

district_codes=['KC', 'KT', 'SSP', 'WTS', 'YTM']
facility_code='TABT'
play_date='2025-03-05'

available_venue_timeslot = fetch_available_venue_timeslot(district_codes, facility_code, play_date)
available_venue_timeslot

{'code': '0',
 'message': 'success',
 'data': {'morning': {'distList': [{'distCode': 'KC',
     'sessionCount': 38,
     'distName': 'Kowloon City',
     'venueList': [{'venueName': 'Fat Kwong Street Sports Centre',
       'venueId': 223,
       'venueImageUrl': 'https://lcsd-smartplay.oss-cn-hongkong.aliyuncs.com/pic/default_facility.jpeg',
       'sessionCount': 6,
       'fatList': [{'fatName': 'Table Tennis Table (A/C)(Urban)',
         'enFatName': 'Table Tennis Table (A/C)(Urban)',
         'fatId': 509,
         'sessionCount': 6,
         'faCodeName': 'Table Tennis',
         'faCode': 'TABT',
         'faGroupCode': 'BAGM',
         'fvrId': 125005431,
         'sessionList': [{'ssnStartTime': '07:00:00',
           'ssnEndTime': '08:00:00',
           'ssnStartDate': '2025-03-05',
           'available': True,
           'peakHour': False,
           'sfadFlag': False,
           'sessionCount': 2},
          {'ssnStartTime': '08:00:00',
           'ssnEndTime': '09:00:00',


In [28]:
for district in available_venue_timeslot["data"]["morning"]["distList"]:
    print(district)

{'distCode': 'KC', 'sessionCount': 38, 'distName': 'Kowloon City', 'venueList': [{'venueName': 'Fat Kwong Street Sports Centre', 'venueId': 223, 'venueImageUrl': 'https://lcsd-smartplay.oss-cn-hongkong.aliyuncs.com/pic/default_facility.jpeg', 'sessionCount': 6, 'fatList': [{'fatName': 'Table Tennis Table (A/C)(Urban)', 'enFatName': 'Table Tennis Table (A/C)(Urban)', 'fatId': 509, 'sessionCount': 6, 'faCodeName': 'Table Tennis', 'faCode': 'TABT', 'faGroupCode': 'BAGM', 'fvrId': 125005431, 'sessionList': [{'ssnStartTime': '07:00:00', 'ssnEndTime': '08:00:00', 'ssnStartDate': '2025-03-05', 'available': True, 'peakHour': False, 'sfadFlag': False, 'sessionCount': 2}, {'ssnStartTime': '08:00:00', 'ssnEndTime': '09:00:00', 'ssnStartDate': '2025-03-05', 'available': True, 'peakHour': False, 'sfadFlag': False, 'sessionCount': 2}, {'ssnStartTime': '09:00:00', 'ssnEndTime': '10:00:00', 'ssnStartDate': '2025-03-05', 'available': True, 'peakHour': False, 'sfadFlag': False, 'sessionCount': 2}, {'ssn

In [ ]:
for district in available_venue_timeslot["data"]["morning"]["distList"]:
    print(district["distCode"])
    # print(district["venueList"])

    for venue in district["venueList"]:
        print(" ", venue["venueName"])
        
        for facility_type in venue["fatList"]:
            print("  ", facility_type["fatName"])

            for time in facility_type["sessionList"]:
                print("   ", time["ssnStartTime"], time["ssnEndTime"], time["available"])
    print("==================================")

KC
  Fat Kwong Street Sports Centre
   Table Tennis Table (A/C)(Urban)
{'ssnStartTime': '07:00:00', 'ssnEndTime': '08:00:00', 'ssnStartDate': '2025-03-05', 'available': True, 'peakHour': False, 'sfadFlag': False, 'sessionCount': 2}
    07:00:00 08:00:00 True
{'ssnStartTime': '08:00:00', 'ssnEndTime': '09:00:00', 'ssnStartDate': '2025-03-05', 'available': True, 'peakHour': False, 'sfadFlag': False, 'sessionCount': 2}
    08:00:00 09:00:00 True
{'ssnStartTime': '09:00:00', 'ssnEndTime': '10:00:00', 'ssnStartDate': '2025-03-05', 'available': True, 'peakHour': False, 'sfadFlag': False, 'sessionCount': 2}
    09:00:00 10:00:00 True
{'ssnStartTime': '10:00:00', 'ssnEndTime': '11:00:00', 'ssnStartDate': '2025-03-05', 'available': False, 'peakHour': False, 'sfadFlag': False, 'sessionCount': 0}
    10:00:00 11:00:00 False
{'ssnStartTime': '11:00:00', 'ssnEndTime': '12:00:00', 'ssnStartDate': '2025-03-05', 'available': False, 'peakHour': False, 'sfadFlag': False, 'sessionCount': 0}
    11:00:00 

In [16]:
import pandas as pd
from datetime import datetime

def process_api_response(api_response):
    """
    Process the LCSD API response into a structured DataFrame of available timeslots
    """
    current_date = datetime(2025, 2, 28)  # Use actual current date in production
    data = []
    
    for period in ['afternoon', 'evening']:
        period_data = api_response.get(period, {}).get('distList', [])
        
        for district in period_data:
            dist_info = {
                'district_code': district['distCode'],
                'district_name': district['distName'].strip(),
                'period': period.capitalize()
            }
            
            for venue in district['venueList']:
                venue_info = {
                    'venue_id': venue['venueId'],
                    'venue_name': venue['venueName']
                }
                
                for facility in venue['fatList']:
                    for session in facility['sessionList']:
                        if session['available']:
                            timeslot_data = {
                                'facility_type': facility['fatName'],
                                'start_time': session['ssnStartTime'],
                                'end_time': session['ssnEndTime'],
                                'available_sessions': session['sessionCount'],
                                'peak_hour': session['peakHour']
                            }
                            
                            # Combine all information
                            record = {**dist_info, **venue_info, **timeslot_data}
                            data.append(record)

    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Convert time columns to datetime
    df['date'] = pd.to_datetime(current_date.date())
    df['start_datetime'] = pd.to_datetime(
        df['date'].astype(str) + ' ' + df['start_time'])
    df['end_datetime'] = pd.to_datetime(
        df['date'].astype(str) + ' ' + df['end_time'])
    
    # Create readable time slot format
    df['timeslot_desc'] = df['start_time'].str[:5] + ' - ' + df['end_time'].str[:5]
    
    # Sort and organize columns
    df = df.sort_values(['district_code', 'venue_id', 'start_datetime'])
    df = df[['district_code', 'district_name', 'venue_id', 'venue_name',
             'date', 'start_datetime', 'end_datetime', 'timeslot_desc',
             'available_sessions', 'peak_hour', 'facility_type']]
    
    # Group by district and timeslot for visualization
    grouped_df = df.groupby(['district_code', 'district_name', 'venue_name', 
                            'timeslot_desc'])['available_sessions'].sum().reset_index()
    
    return df, grouped_df

# Usage example:
full_df, visualization_df = process_api_response(available_venue_timeslot["data"])


In [29]:
def fetch_district_info():
    """Fetches and structures district hierarchy data from the LCSD API."""
    
    # Main response structure
    api_response = {
        "fetch_date": datetime.today().strftime("%Y-%m-%d"),
        "districts": []
    }

    request_headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "Referer": "https://www.smartplay.lcsd.gov.hk/",
        "Accept": "application/json"
    }

    districts_endpoint = "https://www.smartplay.lcsd.gov.hk/rest/param/api/v1/publ/districts/searching-criteria?pgm=N"

    server_response = requests.get(districts_endpoint, headers=request_headers)
    response_data = server_response.json().get("data", 0)

    if response_data:

        for region_group in response_data:
            region_dict = {
                "region_id": region_group["id"],
                "region_code": region_group["code"].strip(),
                "region_name": region_group["name"].strip(),
                "sc_name": region_group["scName"].strip(),
                "tc_name": region_group["tcName"].strip(),
                "subdistricts": []
            }

            for subdistrict in region_group["children"]:
                subdistrict_dict = {
                    "subdistrict_id": subdistrict["id"],
                    "subdistrict_code": subdistrict["code"].strip(),
                    "subdistrict_name": subdistrict["name"].strip(),
                    "en_name": subdistrict["enName"].strip(),
                    "sc_name": subdistrict["scName"].strip(),
                    "tc_name": subdistrict["tcName"].strip()
                }
                region_dict["subdistricts"].append(subdistrict_dict)

            api_response["districts"].append(region_dict)

        return api_response
    
    return {"status": "error", "code": server_response.status_code}

district_info = fetch_district_info()
district_info

{'fetch_date': '2025-02-28',
 'districts': [{'region_id': 202313440,
   'region_code': 'HK',
   'region_name': 'Hong Kong',
   'sc_name': '香港',
   'tc_name': '香港',
   'subdistricts': [{'subdistrict_id': 202311860,
     'subdistrict_code': 'CW',
     'subdistrict_name': 'Central & Western',
     'en_name': 'Central & Western',
     'sc_name': '中西区',
     'tc_name': '中西區'},
    {'subdistrict_id': 202311861,
     'subdistrict_code': 'EN',
     'subdistrict_name': 'Eastern',
     'en_name': 'Eastern',
     'sc_name': '东区',
     'tc_name': '東區'},
    {'subdistrict_id': 202311869,
     'subdistrict_code': 'SN',
     'subdistrict_name': 'Southern',
     'en_name': 'Southern',
     'sc_name': '南区',
     'tc_name': '南區'},
    {'subdistrict_id': 202311873,
     'subdistrict_code': 'WCH',
     'subdistrict_name': 'Wan Chai',
     'en_name': 'Wan Chai',
     'sc_name': '湾仔区',
     'tc_name': '灣仔區'}]},
  {'region_id': 202313441,
   'region_code': 'KLN',
   'region_name': 'Kowloon',
   'sc_name': '九